# CoolingAI Simulator - Interactive Demo

This notebook demonstrates how to use the Physics-Informed Neural Network (PINN) to simulate heat distribution in a data center.

## Table of Contents
1. Setup and Imports
2. Configure Data Center Parameters
3. Train the PINN
4. Visualize Results
5. Interactive Exploration

## 1. Setup and Imports

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add parent directory to path
sys.path.append('..')

from models.trainer import PINNTrainer
from visualization.plotter import HeatVisualizer

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configure matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'MPS' if torch.backends.mps.is_available() else 'CPU'}")

## 2. Configure Data Center Parameters

In [ ]:
# Choose device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(f"Using device: {device}")

# Configuration file path
config_path = '../configs/physics_config.yaml'

# Initialize trainer
trainer = PINNTrainer(config_path, device=device)

print("\n=== Data Center Configuration ===")
print(f"Dimensions: {trainer.config['geometry']['length']}m x {trainer.config['geometry']['width']}m x {trainer.config['geometry']['height']}m")
print(f"Number of server racks: {trainer.config['servers']['num_racks']}")
print(f"Power per rack: {trainer.config['servers']['power_per_rack']} W")
print(f"Inlet temperature: {trainer.config['boundaries']['inlet_temp']} °C")
print(f"\n=== Network Architecture ===")
print(f"Hidden layers: {trainer.config['network']['hidden_layers']}")
print(f"Total parameters: {sum(p.numel() for p in trainer.model.parameters()):,}")

## 3. Train the PINN

**Note:** For quick testing, we'll use fewer epochs. For production, use 10,000+ epochs.

In [ ]:
# Training configuration
EPOCHS = 1000  # Increase to 5000-10000 for better results

print(f"Starting training for {EPOCHS} epochs...\n")

# Train the model
trainer.train(
    epochs=EPOCHS,
    verbose=True,
    checkpoint_dir=None  # Set to '../checkpoints' to save
)

print("\n✓ Training complete!")

## 4. Visualize Training History

In [ ]:
# Plot training loss
trainer.plot_training_history()

## 5. Visualize Temperature Distribution

In [ ]:
# Initialize visualizer
visualizer = HeatVisualizer(trainer.model, trainer.config)

### 5.1 Top-Down View (XY Plane)

In [ ]:
# Plot horizontal slice at mid-height
mid_height = trainer.config['geometry']['height'] / 2

visualizer.plot_2d_slice(
    axis='z',
    slice_value=mid_height,
    t=0.5,
    resolution=150
)

### 5.2 Side View (XZ Plane)

In [ ]:
# Plot vertical slice
mid_width = trainer.config['geometry']['width'] / 2

visualizer.plot_2d_slice(
    axis='y',
    slice_value=mid_width,
    t=0.5,
    resolution=150
)

### 5.3 Multiple Heights

In [ ]:
visualizer.plot_multiple_slices(
    t=0.5,
    n_slices=4,
    axis='z',
    resolution=100
)

### 5.4 Temperature Evolution Over Time

In [ ]:
# Sample point in the center
sample_point = (
    trainer.config['geometry']['length'] / 2,
    trainer.config['geometry']['width'] / 2,
    trainer.config['geometry']['height'] / 2
)

visualizer.plot_time_evolution(
    point=sample_point,
    t_range=(0, 1.0),
    n_points=100
)

## 6. Interactive Exploration

In [ ]:
# Interactive widget for exploring temperature at different locations
from ipywidgets import interact, FloatSlider

geom = trainer.config['geometry']

@interact(
    x=FloatSlider(min=0, max=geom['length'], step=0.5, value=5.0, description='X (m):'),
    y=FloatSlider(min=0, max=geom['width'], step=0.5, value=4.0, description='Y (m):'),
    z=FloatSlider(min=0, max=geom['height'], step=0.1, value=1.5, description='Z (m):'),
    t=FloatSlider(min=0, max=1.0, step=0.1, value=0.5, description='Time (s):')
)
def predict_temperature(x, y, z, t):
    T = trainer.model.predict_temperature(
        np.array([x]), np.array([y]), np.array([z]), np.array([t])
    )[0]
    print(f"\nLocation: ({x:.1f}m, {y:.1f}m, {z:.1f}m) at t={t:.1f}s")
    print(f"Predicted Temperature: {T:.2f} °C")
    return T

## 7. Sample Predictions

In [ ]:
# Sample multiple points
sample_locations = [
    (1.0, 1.0, 0.5, "Near inlet, floor level"),
    (5.0, 4.0, 1.5, "Center of room, mid-height"),
    (9.0, 7.0, 2.5, "Far corner, near ceiling"),
]

t_eval = 0.5  # 0.5 seconds

print(f"\n=== Temperature Predictions at t={t_eval}s ===")
print("=" * 60)

for x, y, z, description in sample_locations:
    T = trainer.model.predict_temperature(
        np.array([x]), np.array([y]), np.array([z]), np.array([t_eval])
    )[0]
    
    print(f"\nLocation: ({x:.1f}m, {y:.1f}m, {z:.1f}m)")
    print(f"Description: {description}")
    print(f"Temperature: {T:.2f} °C")

print("\n" + "=" * 60)

## 8. Save the Model (Optional)

In [ ]:
# Uncomment to save the trained model
# from pathlib import Path
# checkpoint_dir = Path('../checkpoints')
# checkpoint_dir.mkdir(exist_ok=True)
# trainer.save_checkpoint(checkpoint_dir / 'notebook_model.pt', epoch=EPOCHS)
# print(f"✓ Model saved to {checkpoint_dir / 'notebook_model.pt'}")

## Next Steps

1. **Increase training epochs** to 5,000-10,000 for better accuracy
2. **Modify configuration** in `configs/physics_config.yaml`
3. **Experiment with different geometries** and server configurations
4. **Add more boundary conditions** (outlets, walls with heat transfer)
5. **Incorporate air flow** for more realistic simulations
6. **Validate against experimental data** if available

See `QUICKSTART.md` for more details!